In [2]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM


model_name = "Qwen/Qwen2.5-Math-1.5B"  # Or any other supported model

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
)

# Save to local path
save_path = "/assignment5-alignment/models/local-qwen-2_5-math-1_5B"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

('/assignment5-alignment/models/local-qwen-2_5-math-1_5B/tokenizer_config.json',
 '/assignment5-alignment/models/local-qwen-2_5-math-1_5B/special_tokens_map.json',
 '/assignment5-alignment/models/local-qwen-2_5-math-1_5B/vocab.json',
 '/assignment5-alignment/models/local-qwen-2_5-math-1_5B/merges.txt',
 '/assignment5-alignment/models/local-qwen-2_5-math-1_5B/added_tokens.json',
 '/assignment5-alignment/models/local-qwen-2_5-math-1_5B/tokenizer.json')

## vLLM Inference

In [1]:
import os
from vllm import LLM, SamplingParams

INFO 08-04 10:43:31 __init__.py:190] Automatically detected platform cuda.


In [2]:
# Create a sampling params object, stopping generation on newline.
sampling_params = SamplingParams(
    temperature=1.0, top_p=1.0, max_tokens=1024, stop=["\n"]
)
save_path = "/assignment5-alignment/models/local-qwen-2_5-math-1_5B"

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# Create an LLM.
llm = LLM(model=save_path)

INFO 08-04 10:43:42 config.py:542] This model supports multiple tasks: {'score', 'classify', 'embed', 'reward', 'generate'}. Defaulting to 'generate'.
INFO 08-04 10:43:42 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='/assignment5-alignment/models/local-qwen-2_5-math-1_5B', speculative_config=None, tokenizer='/assignment5-alignment/models/local-qwen-2_5-math-1_5B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), se

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 08-04 10:43:48 model_runner.py:1115] Loading model weights took 2.8797 GB
INFO 08-04 10:43:48 worker.py:267] Memory profiling takes 0.53 seconds
INFO 08-04 10:43:48 worker.py:267] the current vLLM instance can use total_gpu_memory (79.10GiB) x gpu_memory_utilization (0.90) = 71.19GiB
INFO 08-04 10:43:48 worker.py:267] model weights take 2.88GiB; non_torch_memory takes 0.15GiB; PyTorch activation peak memory takes 1.42GiB; the rest of the memory reserved for KV Cache is 66.73GiB.
INFO 08-04 10:43:48 executor_base.py:110] # CUDA blocks: 156196, # CPU blocks: 9362
INFO 08-04 10:43:48 executor_base.py:115] Maximum concurrency for 4096 tokens per request: 610.14x
INFO 08-04 10:43:51 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:27<00:00,  1.28it/s]

INFO 08-04 10:44:19 model_runner.py:1562] Graph capturing finished in 27 secs, took 0.29 GiB
INFO 08-04 10:44:19 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 31.12 seconds


In [5]:
# Sample prompts.
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
    "Please help me to solve equation 1 + 1 = ? Answer: 1 + 1 = 2\nWhich capital of GB? Answer: Capital is London\nWhich is the most popular chess opening? Answer:"
]

In [6]:
# Generate texts from the prompts. The output is a list of RequestOutput objects
# that contain the prompt, generated text, and other information.
outputs = llm.generate(prompts, sampling_params)

Processed prompts: 100%|██████████| 5/5 [00:00<00:00, 13.94it/s, est. speed input: 186.95 toks/s, output: 753.35 toks/s]


In [7]:
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")


Prompt: 'Hello, my name is', Generated text: ' K deadly. I have to find out what I fall into as my position. Then, I must apply the force on the roll.'
Prompt: 'The president of the United States is', Generated text: ' planning to distribute a certain number of kits to schools around the world to enhance their understanding of international relations. Each kit contains materials for a discussion about how nations can leverage their unique economic, political, and social resources for growth. If the president has a total of 15 kits and wants to distribute them equally among 3 schools, how many kits will each school receive?'
Prompt: 'The capital of France is', Generated text: ' located at 9.9850 in the east-north map projection. If a cyclist decides to ride from a point 0.0072 in the north direction and arrives at a capital, how much farther did the cyclist need to travel to cover the full distance from the starting point to the capital? To determine the total distance the cyclist needs

## GSM8k eval

In [8]:
import re
import json
from typing import List, Callable
from pathlib import Path

from cs336_alignment.drgrpo_grader import r1_zero_reward_fn

/assignment5-alignment/cs336_alignment/drgrpo_grader.py:45: SyntaxWarning: invalid escape sequence '\{'
  m = re.search("^\\\\text\{(?P<text>.+?)\}$", answer)
/assignment5-alignment/cs336_alignment/drgrpo_grader.py:320: SyntaxWarning: invalid escape sequence '\%'
  string = string.replace("\%", "")
/assignment5-alignment/cs336_alignment/drgrpo_grader.py:673: SyntaxWarning: invalid escape sequence '\^'
  BAD_REGEXES = ["\^[0-9]+\^", "\^[0-9][0-9]+"]
/assignment5-alignment/cs336_alignment/drgrpo_grader.py:673: SyntaxWarning: invalid escape sequence '\^'
  BAD_REGEXES = ["\^[0-9]+\^", "\^[0-9][0-9]+"]
/assignment5-alignment/cs336_alignment/drgrpo_grader.py:753: SyntaxWarning: invalid escape sequence '\d'
  p1 = re.compile("(\d)(,)(\d\d\d)($|\D)")
/assignment5-alignment/cs336_alignment/drgrpo_grader.py:768: SyntaxWarning: invalid escape sequence '\{'
  m = re.search("^\\\\text\{(?P<text>.+?)\}$", expr)
/assignment5-alignment/cs336_alignment/drgrpo_grader.py:801: SyntaxWarning: invalid esca

In [9]:
with open("/assignment5-alignment/data/gsm8k/test.jsonl", "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

prompt_template = Path("/assignment5-alignment/cs336_alignment/prompts/r1_zero.prompt").read_text()

gsm8k_pattern = r"####\s*(.*)"

In [10]:
print(prompt_template.format(**data[0]))

A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer. The reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> answer here </answer>.
User: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
Assistant: <think>


In [11]:
match = re.search(r"####\s*(.*)", data[0]["answer"])
if match:
    answer = match.group(1).strip()
    print(f"Final answer: {answer}")

Final answer: 18


In [12]:
prompts = []
answers = []

for i, line in enumerate(data):
    prompts.append(
        prompt_template.format(**line)
    )
    match = re.search(r"####\s*(.*)", line["answer"])
    if match:
        answer = match.group(1).strip()
        answers.append(answer)
    else:
        raise ValueError(f"Wrong answer format! Index: {i}")


In [13]:
prompts[:10]

["A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer. The reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> answer here </answer>.\nUser: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?\nAssistant: <think>",
 'A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer. The reasoning process is enclosed within <think> </think> and a

In [14]:
answers[:10]

['18', '3', '70000', '540', '20', '64', '260', '160', '45', '460']

In [15]:
eval_sampling_params = SamplingParams(
    temperature=1.0, 
    top_p=1.0,
    max_tokens=1024,
    stop=["</answer>"],
    include_stop_str_in_output=True,
)

In [16]:
outputs = llm.generate(prompts[:10], eval_sampling_params)

generated_texts = []

for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    generated_texts.append(generated_text)
    print(f"Generated text: {generated_text!r}")


Processed prompts: 100%|██████████| 10/10 [00:03<00:00,  2.79it/s, est. speed input: 433.44 toks/s, output: 883.32 toks/s]

Generated text: ' user lay 16 eggs per day. user eats three for breakfast every morning and bakes muffins for her friends every day with four. user sells the remainder at the farmers\' market daily for 2 dollars per fresh duck egg. how much in dollars does she make per day?</think>\n<answer> <answer type="block"> Shark已经被.smith 2012/09/06 21:00 PM </answer>'
Generated text: "If the blue fiber requires 2 bolts, let's assume the white fiber requires 'x' bolts. Since the white fiber is half that of the blue fiber, we have x = 1/2 * 2. We need to find the total number of bolts. The total number of bolts is calculated by adding the blue and white fibers together. As the blue fiber requires 2 bolts, the white fiber will require half of that, which is 1 bolt. The total number of bolts is 2 + 1, as we are adding the number of blue fibers (2) to twice the number of white fibers (1). </think>\nThe answer is\n{answer}\n <answer> and {answer}. </answer>"
Generated text: 'The price of the house is 

In [17]:
r1_zero_reward_fn(generated_texts[0], answers[0])

{'format_reward': 0.0, 'answer_reward': 0.0, 'reward': 0.0}

In [18]:
def evaluate_vllm(
    vllm_model: LLM,
    reward_fn: Callable[[str, str], dict[str, float]],
    prompts: List[str],
    eval_sampling_params: SamplingParams
) -> List[dict[str, float]]:
    """
    Evaluate a language model on a list of prompts,
    compute evaluation metrics, and serialize results to disk.
    """
    outputs = vllm_model.generate(prompts, eval_sampling_params)

    results = []
    for output in outputs:
        generated_text = output.outputs[0].text
        results.append(
            reward_fn(generated_text, generated_text)
        )
    
    return results, outputs


In [19]:
results, outputs = evaluate_vllm(
    vllm_model=llm,
    reward_fn=r1_zero_reward_fn,
    prompts=prompts,
    eval_sampling_params=eval_sampling_params,
)

Processed prompts: 100%|██████████| 1319/1319 [00:29<00:00, 45.41it/s, est. speed input: 6994.59 toks/s, output: 10347.82 toks/s]


In [19]:
format_reward = 0
answer_reward = 0
reward = 0

for result in results:
    format_reward += result["format_reward"] / len(results)
    answer_reward += result["answer_reward"] / len(results)
    reward += result["reward"] / len(results)

print(f"format_reward: {format_reward}")
print(f"answer_reward: {answer_reward}")
print(f"reward: {reward}")

format_reward: 0.1721000758150113
answer_reward: 0.002274450341167551
reward: 0.002274450341167551


In [20]:
with open("/assignment5-alignment/data/base_vllm_results.json", "w+") as f:
    json.dump(results, f)
